In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

import sqlite3
from bk_modeling import *
import scipy.stats as stats
import sklearn.linear_model as linear

pd.options.display.float_format = '{:,.2f}'.format
sns.set(style="whitegrid")

## Predictions

Now that the model is built, the front office is able to bring us players that they want salary esitmates for. As the list of free agents for 2022 is [already available](https://www.spotrac.com/nba/free-agents/), the team has given us the following players to look into:

**Montrez Harrell**

**Dennis Schroder**

**Pat Connaughton**

### Loading Up Model

Before beginning, I will load up the data from the data base and get the necessary features for the model.

In [2]:
con = sqlite3.connect('data.db')

In [3]:
query = """
SELECT 
    p.name, c.year, p.position, p.height, p.weight,
    c.age, c.year-p.rookie_year,
    s.games, s.minutes, s.pts, s.trb, s.ast, s.stl, s.blk, s.tov, s.fg, s.fg_per, s.three_fg, s.three_fg_per,
    s.efg_per, s.ft_per, 
    c.type, c.chg_team, c.length, c.avg_dollars
FROM contracts c
JOIN players p
    ON c.id=p.id
JOIN stats s
    ON c.id=s.id AND c.year=s.year
"""

contracts = pd.DataFrame(con.execute(query),
                         columns=['player', 'year', 'position', 'height', 'weight',
                                  'age', 'years_exp',
                                  'games', 'min', 'pts', 'rbs', 'ast', 'stl', 'blk', 'tov', 'fg', 'fg_per',
                                  'three_fg', 'three_per', 'efg_per', 'ft_per',
                                  'type', 'chg_team', 'length', 'avg_dollars'])

First, I will get the dummy variables for `type`.

In [4]:
contracts = pd.concat([contracts, pd.get_dummies(contracts['type'])], axis=1) 

Next, I will get the squared variables.

In [5]:
contracts['pts_sq'] = contracts['pts']**2
contracts['min_sq'] = contracts['min']**2
contracts['rbs_sq'] = contracts['rbs']**2
contracts['ast_sq'] = contracts['ast']**2
contracts['stl_sq'] = contracts['stl']**2
contracts['blk_sq'] = contracts['blk']**2
contracts['tov_sq'] = contracts['tov']**2
contracts['three_fg_sq'] = contracts['three_fg']**2

Next, I will get divide the players into short, medium, and tall groups and compute the necessary interaction terms.

In [6]:
contracts['short'] = [1 if x<=76 else 0 for x in contracts['height']]
contracts['tall'] = [1 if x>=82 else 0 for x in contracts['height']]
contracts['medium'] = [1 if x[0]+x[1] != 1 else 0 for x in zip(contracts['short'], contracts['tall'])]

In [7]:
contracts['short:pts'] = contracts['short'] * contracts['pts']
contracts['short:rbs'] = contracts['short'] * contracts['rbs']
contracts['short:blk'] = contracts['short'] * contracts['blk']
contracts['short:efg_per'] = contracts['short'] * contracts['efg_per']
contracts['short:ast'] = contracts['short'] * contracts['ast']
contracts['short:stl'] = contracts['short'] * contracts['stl']
contracts['short:three_fg'] = contracts['short'] * contracts['three_fg']

contracts['tall:pts'] = contracts['tall'] * contracts['pts']
contracts['tall:rbs'] = contracts['tall'] * contracts['rbs']
contracts['tall:blk'] = contracts['tall'] * contracts['blk']
contracts['tall:efg_per'] = contracts['tall'] * contracts['efg_per']
contracts['tall:ast'] = contracts['tall'] * contracts['ast']
contracts['tall:stl'] = contracts['tall'] * contracts['stl']
contracts['tall:three_fg'] = contracts['tall'] * contracts['three_fg']

And lastly, creating binary variables about the length of the contract.

In [8]:
contracts['one_yr'] = [1 if x==1 else 0 for x in contracts['length']]
contracts['four_five_yr'] = [1 if x>=4 else 0 for x in contracts['length']]
contracts['two_three_yr'] = [1 if x[0]+x[1] != 1 else 0 for x in zip(contracts['one_yr'], contracts['four_five_yr'])]

In [9]:
y = 'avg_dollars'
X = ['height', 'age', 'min_sq', 'pts_sq',
     'rbs_sq', 'ast_sq', 'stl_sq', 'blk_sq', 'tov_sq', 'three_fg_sq',
     'RFA', 'chg_team', 'two_three_yr', 'four_five_yr', 'length',
     'short:pts', 'short:rbs', 'short:blk', 'short:efg_per',
     'short:ast', 'short:stl', 'short:three_fg',
     'tall:pts', 'tall:rbs', 'tall:blk', 'tall:efg_per',
     'tall:ast', 'tall:stl', 'tall:three_fg']

And now I will fit the data to the model and get a result.

In [10]:
result = bootstrap_linear_reg(contracts, X, y)

### Montrez Harrell

Montrez Harrell is an interesting player currently playing for the Washington Wizards. Montrez Harrell won the 6th Man of the Year award in 2020, but had a really poor showing in the 2020 playoffs, with many media members critizing his defensive ability. After an up and down 2021 season, Montrez Harrell was traded to the Washington Wizards. Set to enter free agency in 2022, Montrez Harrell currently has the following stats for this season [(via BasketballReference)](https://www.basketball-reference.com/players/h/harremo01.html)

Min: 26.1
<br>Pts: 14.9
<br>Rbs: 7.9
<br>Ast: 2.1
<br>Stl: 0.6
<br>Blk: 0.7
<br>Tov: 1.3
<br>Three_FG: 0.1
<br>eFG_per: 0.647

Harrell is 6'7 (79 inches) and will be 28.5 when free agency happens next summer. He will be an Unrestricted Free Agent and is said to be looking for a 3 year contract.

I will start by creating a new DataFrame `free_agents` and adding Harrell to it. 

In [11]:
free_agents = pd.DataFrame(data={}, columns=['name', 'height', 'age', 'min', 'pts', 'rbs', 'ast', 'stl',
                                             'blk', 'tov', 'three_fg', 'efg_per'])

In [12]:
free_agents.loc[0] = ['Montrez Harrell', 79, 28.5, 26.1, 14.9, 7.9, 2.1, 0.6, 0.7, 1.3, 0.1, 0.647]

In [13]:
free_agents.head()

,name,height,age,min,pts,rbs,ast,stl,blk,tov,three_fg,efg_per
0,Montrez Harrell,79,28.50,26.10,14.90,7.90,2.10,0.60,0.70,1.30,0.10,0.65


Now, I will create the necessary features for the model. First I will get the squared features

In [14]:
free_agents['pts_sq'] = free_agents['pts']**2
free_agents['min_sq'] = free_agents['min']**2
free_agents['rbs_sq'] = free_agents['rbs']**2
free_agents['ast_sq'] = free_agents['ast']**2
free_agents['stl_sq'] = free_agents['stl']**2
free_agents['blk_sq'] = free_agents['blk']**2
free_agents['tov_sq'] = free_agents['tov']**2
free_agents['three_fg_sq'] = free_agents['three_fg']**2

Next, I will decide which height group the player falls into and compute the various interaction terms.

In [15]:
free_agents['short'] = [1 if x<=76 else 0 for x in free_agents['height']]
free_agents['tall'] = [1 if x>=82 else 0 for x in free_agents['height']]
free_agents['medium'] = [1 if x[0]+x[1] != 1 else 0 for x in zip(free_agents['short'], free_agents['tall'])]

In [16]:
free_agents['short:pts'] = free_agents['short'] * free_agents['pts']
free_agents['short:rbs'] = free_agents['short'] * free_agents['rbs']
free_agents['short:blk'] = free_agents['short'] * free_agents['blk']
free_agents['short:efg_per'] = free_agents['short'] * free_agents['efg_per']
free_agents['short:ast'] = free_agents['short'] * free_agents['ast']
free_agents['short:stl'] = free_agents['short'] * free_agents['stl']
free_agents['short:three_fg'] = free_agents['short'] * free_agents['three_fg']

free_agents['tall:pts'] = free_agents['tall'] * free_agents['pts']
free_agents['tall:rbs'] = free_agents['tall'] * free_agents['rbs']
free_agents['tall:blk'] = free_agents['tall'] * free_agents['blk']
free_agents['tall:efg_per'] = free_agents['tall'] * free_agents['efg_per']
free_agents['tall:ast'] = free_agents['tall'] * free_agents['ast']
free_agents['tall:stl'] = free_agents['tall'] * free_agents['stl']
free_agents['tall:three_fg'] = free_agents['tall'] * free_agents['three_fg']

Next, I will set `RFA` and `chg_team`.

In [17]:
free_agents['RFA'] = 0
free_agents['chg_team'] = 1

Lastly, I will define the length and length binary features for Montrez.

In [18]:
free_agents['length'] = 3
free_agents['two_three_yr'] = 1
free_agents['four_five_yr'] = 0

In [19]:
free_agents.head()

,name,height,age,min,pts,rbs,ast,stl,blk,tov,...,tall:blk,tall:efg_per,tall:ast,tall:stl,tall:three_fg,RFA,chg_team,length,two_three_yr,four_five_yr
0,Montrez Harrell,79,28.50,26.10,14.90,7.90,2.10,0.60,0.70,1.30,...,0.00,0.00,0.00,0.00,0.00,0,1,3,1,0


Now, that Harrell's feature values have been calculated, I will predict his upcoming salary using the model.

In [20]:
m_harrell = [x for x in free_agents.loc[0][X].values]

In [21]:
predict(result, [m_harrell])

[array([15117095.02955979])]

The model predicts that Harrell will make just over 15 million dollars next season. This is an upgrade over the 9.7 million he is making this year. So far this season, Harrell is having a very good year, which explains the increase in pay from his last contract.

### Dennis Schroder

Dennis Schroder finished second in 6th Man of the Year voting to Montrez Harrell in 2020, but was traded to the Los Angeles Lakers in the offseason. In an now famous story, Schroder turned down a contract extension with the Lakers reportedly worth 21 million dollars annually over 4 years in hopes of earning more in free agency. However, the big offer never came and Schroder was forced to sign a 5.9 million dollar offer with the Boston Celtics. The reason Schroder chose the Celtics is that he thought the playing time and role the Celtics were offering him would allow him to rebuild his value to reenter free agency in 2022. So far in the season, here are Schroder's stats:

Min: 33.4
<br>Pts: 17.5
<br>Rbs: 3.6
<br>Ast: 4.9
<br>Stl: 0.9
<br>Blk: 0.0
<br>Tov: 2.6
<br>Three_FG: 1.6
<br>eFG_per: 0.500

Harrell is 6'3 (75 inches) and will be 28.8 when free agency happens next summer. He will be an Unrestricted Free Agent and is said to be looking for a 4 year contract.

I will start by adding Schroder to the `free_agents` DataFrame. 

In [22]:
free_agents.loc[1] = ['Dennis Schoder', 75, 28.8, 33.4, 17.5, 3.6, 4.9, 0.9, 0.0, 2.6, 1.6, 0.5] + 30*[0]

In [23]:
free_agents.head()

,name,height,age,min,pts,rbs,ast,stl,blk,tov,...,tall:blk,tall:efg_per,tall:ast,tall:stl,tall:three_fg,RFA,chg_team,length,two_three_yr,four_five_yr
0,Montrez Harrell,79,28.50,26.10,14.90,7.90,2.10,0.60,0.70,1.30,...,0.00,0.00,0.00,0.00,0.00,0,1,3,1,0
1,Dennis Schoder,75,28.80,33.40,17.50,3.60,4.90,0.90,0.00,2.60,...,0.00,0.00,0.00,0.00,0.00,0,0,0,0,0


Now, I will create the necessary features for Schroder. First I will get the squared features.

In [24]:
free_agents.at[1,'pts_sq'] = free_agents.at[1,'pts']**2
free_agents.at[1,'min_sq'] = free_agents.at[1,'min']**2
free_agents.at[1,'rbs_sq'] = free_agents.at[1,'rbs']**2
free_agents.at[1,'ast_sq'] = free_agents.at[1,'ast']**2
free_agents.at[1,'stl_sq'] = free_agents.at[1,'stl']**2
free_agents.at[1,'blk_sq'] = free_agents.at[1,'blk']**2
free_agents.at[1,'tov_sq'] = free_agents.at[1,'tov']**2
free_agents.at[1,'three_fg_sq'] = free_agents.at[1,'three_fg']**2

Next, I will decide which height group Schroder falls into and compute the various interaction terms.

In [25]:
if free_agents.at[1,'height']<=76:
    free_agents.at[1,'short']=1
    free_agents.at[1,'medium']=0
    free_agents.at[1,'tall']=0
elif free_agents.at[1,'height']<=81:
    free_agents.at[1,'short']=0
    free_agents.at[1,'medium']=1
    free_agents.at[1,'tall']=0
else:
    free_agents.at[1,'short']=0
    free_agents.at[1,'medium']=0
    free_agents.at[1,'tall']=1

In [26]:
free_agents.at[1,'short:pts'] = free_agents.at[1,'short'] * free_agents.at[1,'pts']
free_agents.at[1,'short:rbs'] = free_agents.at[1,'short'] * free_agents.at[1,'rbs']
free_agents.at[1,'short:blk'] = free_agents.at[1,'short'] * free_agents.at[1,'blk']
free_agents.at[1,'short:efg_per'] = free_agents.at[1,'short'] * free_agents.at[1,'efg_per']
free_agents.at[1,'short:ast'] = free_agents.at[1,'short'] * free_agents.at[1,'ast']
free_agents.at[1,'short:stl'] = free_agents.at[1,'short'] * free_agents.at[1,'stl']
free_agents.at[1,'short:three_fg'] = free_agents.at[1,'short'] * free_agents.at[1,'three_fg']

free_agents.at[1,'tall:pts'] = free_agents.at[1,'tall'] * free_agents.at[1,'pts']
free_agents.at[1,'tall:rbs'] = free_agents.at[1,'tall'] * free_agents.at[1,'rbs']
free_agents.at[1,'tall:blk'] = free_agents.at[1,'tall'] * free_agents.at[1,'blk']
free_agents.at[1,'tall:efg_per'] = free_agents.at[1,'tall'] * free_agents.at[1,'efg_per']
free_agents.at[1,'tall:ast'] = free_agents.at[1,'tall'] * free_agents.at[1,'ast']
free_agents.at[1,'tall:stl'] = free_agents.at[1,'tall'] * free_agents.at[1,'stl']
free_agents.at[1,'tall:three_fg'] = free_agents.at[1,'tall'] * free_agents.at[1,'three_fg']

Next, I will set `RFA` and `chg_team`.

In [27]:
free_agents.at[1,'RFA'] = 0
free_agents.at[1,'chg_team'] = 1

Lastly, I will define the length and length binary features for Dennis.

In [28]:
free_agents.at[1,'length'] = 4
free_agents.at[1,'two_three_yr'] = 0
free_agents.at[1,'four_five_yr'] = 1

In [29]:
free_agents.head()

,name,height,age,min,pts,rbs,ast,stl,blk,tov,...,tall:blk,tall:efg_per,tall:ast,tall:stl,tall:three_fg,RFA,chg_team,length,two_three_yr,four_five_yr
0,Montrez Harrell,79,28.50,26.10,14.90,7.90,2.10,0.60,0.70,1.30,...,0.00,0.00,0.00,0.00,0.00,0,1,3,1,0
1,Dennis Schoder,75,28.80,33.40,17.50,3.60,4.90,0.90,0.00,2.60,...,0.00,0.00,0.00,0.00,0.00,0,1,4,0,1


Now, that Schroder's feature values have been calculated, I will predict his upcoming salary using the model.

In [30]:
d_schroder = [x for x in free_agents.loc[1][X].values]

In [31]:
predict(result, [d_schroder])

[array([17356858.80622774])]

The model predicts that Schroder will make roughly 17.3 million dollars per year for the next four years. This is not quite the 21 million dollars per year that he turned down, but it is still a lot of money!

### Pat Connaughton

While both Montrez Harrell and Dennis Schroder are really good players, they are not known for their shooting. Harrell is known for his inside game and toughness and Schroder is known for his playmaking and quickness. In today's NBA, you also need guys who can knock down threes. Enter Pat Connaughton. Pat Connaughton shot 37% from three in the 2020-21 season, but is currently shooting 43% to start the 2021-2022 season. If he keeps this pace up, he could be in line for a nice new contract in the offseason. Let's see what the model says about Connaughton. Here are his statistics at the current moment.

Min: 28.8
<br>Pts: 12.1
<br>Rbs: 4.6
<br>Ast: 1.3
<br>Stl: 1.0
<br>Blk: 0.4
<br>Tov: 0.8
<br>Three_FG: 2.7
<br>eFG_per: 0.653

Harrell is 6'5 (77 inches) and will be 29.5 when free agency happens next summer. He will be an Unrestricted Free Agent and is said to be looking for a 4 year contract.

I will start by adding Connaughton to the `free_agents` DataFrame. 

In [32]:
free_agents.loc[2] = ['Pat Connaughton', 77, 29.5, 28.8, 12.1, 4.6, 1.3, 1.0, 0.4, 0.8, 2.7, 0.653] + 30*[0]

In [33]:
free_agents

,name,height,age,min,pts,rbs,ast,stl,blk,tov,...,tall:blk,tall:efg_per,tall:ast,tall:stl,tall:three_fg,RFA,chg_team,length,two_three_yr,four_five_yr
0,Montrez Harrell,79,28.50,26.10,14.90,7.90,2.10,0.60,0.70,1.30,...,0.00,0.00,0.00,0.00,0.00,0,1,3,1,0
1,Dennis Schoder,75,28.80,33.40,17.50,3.60,4.90,0.90,0.00,2.60,...,0.00,0.00,0.00,0.00,0.00,0,1,4,0,1
2,Pat Connaughton,77,29.50,28.80,12.10,4.60,1.30,1.00,0.40,0.80,...,0.00,0.00,0.00,0.00,0.00,0,0,0,0,0


Now, I will create the necessary features for Connaughton. First I will get the squared features.

In [34]:
free_agents.at[2,'pts_sq'] = free_agents.at[2,'pts']**2
free_agents.at[2,'min_sq'] = free_agents.at[2,'min']**2
free_agents.at[2,'rbs_sq'] = free_agents.at[2,'rbs']**2
free_agents.at[2,'ast_sq'] = free_agents.at[2,'ast']**2
free_agents.at[2,'stl_sq'] = free_agents.at[2,'stl']**2
free_agents.at[2,'blk_sq'] = free_agents.at[2,'blk']**2
free_agents.at[2,'tov_sq'] = free_agents.at[2,'tov']**2
free_agents.at[2,'three_fg_sq'] = free_agents.at[2,'three_fg']**2

Next, I will decide which height group Connaughton falls into and compute the various interaction terms.

In [35]:
if free_agents.at[2,'height']<=76:
    free_agents.at[2,'short']=1
    free_agents.at[2,'medium']=0
    free_agents.at[2,'tall']=0
elif free_agents.at[2,'height']<=81:
    free_agents.at[2,'short']=0
    free_agents.at[2,'medium']=1
    free_agents.at[2,'tall']=0
else:
    free_agents.at[2,'short']=0
    free_agents.at[2,'medium']=0
    free_agents.at[2,'tall']=1

In [36]:
free_agents.at[2,'short:pts'] = free_agents.at[2,'short'] * free_agents.at[2,'pts']
free_agents.at[2,'short:rbs'] = free_agents.at[2,'short'] * free_agents.at[2,'rbs']
free_agents.at[2,'short:blk'] = free_agents.at[2,'short'] * free_agents.at[2,'blk']
free_agents.at[2,'short:efg_per'] = free_agents.at[2,'short'] * free_agents.at[2,'efg_per']
free_agents.at[2,'short:ast'] = free_agents.at[2,'short'] * free_agents.at[2,'ast']
free_agents.at[2,'short:stl'] = free_agents.at[2,'short'] * free_agents.at[2,'stl']
free_agents.at[2,'short:three_fg'] = free_agents.at[2,'short'] * free_agents.at[2,'three_fg']

free_agents.at[2,'tall:pts'] = free_agents.at[2,'tall'] * free_agents.at[2,'pts']
free_agents.at[2,'tall:rbs'] = free_agents.at[2,'tall'] * free_agents.at[2,'rbs']
free_agents.at[2,'tall:blk'] = free_agents.at[2,'tall'] * free_agents.at[2,'blk']
free_agents.at[2,'tall:efg_per'] = free_agents.at[2,'tall'] * free_agents.at[2,'efg_per']
free_agents.at[2,'tall:ast'] = free_agents.at[2,'tall'] * free_agents.at[2,'ast']
free_agents.at[2,'tall:stl'] = free_agents.at[2,'tall'] * free_agents.at[2,'stl']
free_agents.at[2,'tall:three_fg'] = free_agents.at[2,'tall'] * free_agents.at[2,'three_fg']

Next, I will set `RFA` and `chg_team`.

In [37]:
free_agents.at[2,'RFA'] = 0
free_agents.at[2,'chg_team'] = 1

Lastly, I will define the length and length binary features for Pat.

In [38]:
free_agents.at[2,'length'] = 4
free_agents.at[2,'two_three_yr'] = 0
free_agents.at[2,'four_five_yr'] = 1

In [39]:
free_agents.head()

,name,height,age,min,pts,rbs,ast,stl,blk,tov,...,tall:blk,tall:efg_per,tall:ast,tall:stl,tall:three_fg,RFA,chg_team,length,two_three_yr,four_five_yr
0,Montrez Harrell,79,28.50,26.10,14.90,7.90,2.10,0.60,0.70,1.30,...,0.00,0.00,0.00,0.00,0.00,0,1,3,1,0
1,Dennis Schoder,75,28.80,33.40,17.50,3.60,4.90,0.90,0.00,2.60,...,0.00,0.00,0.00,0.00,0.00,0,1,4,0,1
2,Pat Connaughton,77,29.50,28.80,12.10,4.60,1.30,1.00,0.40,0.80,...,0.00,0.00,0.00,0.00,0.00,0,1,4,0,1


Now, that Connaughton's feature values have been calculated, I will predict his upcoming salary using the model.

In [40]:
p_connaughton = [x for x in free_agents.loc[2][X].values]

In [41]:
predict(result, [p_connaughton])

[array([15272202.66749128])]

The model predicts that Connaughton will make just over 15 million dollars next year, which is about 10 million more than he is making this season. This might be an example of one of the very high miscalculations by the model, but as the contract has not been signed yet, there is no way to know that at this point.

The front office does not want to spend that much money on Connaughton, especially when he has not really proved that level of shooting until this season. They are still interested in guards who can shoot to round out the team and have asked me to find a player like this who might be a little cheaper.

### Bryn Forbes

One option to target instead of Connaughton would be Bryn Forbes. Forbes currently is a SG for the San Antonio Spurs and is shooting 42.5% from three so far, albeit on lower attempts. Here are his other statistical numbers.

Min: 14.3
<br>Pts: 7.2
<br>Rbs: 1.2
<br>Ast: 0.7
<br>Stl: 0.4
<br>Blk: 0.0
<br>Tov: 0.9
<br>Three_FG: 1.2
<br>eFG_per: 0.564

Harrell is 6'2 (74 inches) and will be 29 when free agency happens next summer. He will be an Unrestricted Free Agent and is said to be looking for a 2 year contract.

I will start by adding Forbes to the `free_agents` DataFrame. 

In [42]:
free_agents.loc[3] = ['Bryn Forbes', 74, 29.0, 14.3, 7.2, 1.2, 0.7, 0.4, 0.0, 0.9, 1.2, 0.564] + 30*[0]

In [43]:
free_agents

,name,height,age,min,pts,rbs,ast,stl,blk,tov,...,tall:blk,tall:efg_per,tall:ast,tall:stl,tall:three_fg,RFA,chg_team,length,two_three_yr,four_five_yr
0,Montrez Harrell,79,28.50,26.10,14.90,7.90,2.10,0.60,0.70,1.30,...,0.00,0.00,0.00,0.00,0.00,0,1,3,1,0
1,Dennis Schoder,75,28.80,33.40,17.50,3.60,4.90,0.90,0.00,2.60,...,0.00,0.00,0.00,0.00,0.00,0,1,4,0,1
2,Pat Connaughton,77,29.50,28.80,12.10,4.60,1.30,1.00,0.40,0.80,...,0.00,0.00,0.00,0.00,0.00,0,1,4,0,1
3,Bryn Forbes,74,29.00,14.30,7.20,1.20,0.70,0.40,0.00,0.90,...,0.00,0.00,0.00,0.00,0.00,0,0,0,0,0


Now, I will create the necessary features for Forbes. First I will get the squared features.

In [44]:
free_agents.at[3,'pts_sq'] = free_agents.at[3,'pts']**2
free_agents.at[3,'min_sq'] = free_agents.at[3,'min']**2
free_agents.at[3,'rbs_sq'] = free_agents.at[3,'rbs']**2
free_agents.at[3,'ast_sq'] = free_agents.at[3,'ast']**2
free_agents.at[3,'stl_sq'] = free_agents.at[3,'stl']**2
free_agents.at[3,'blk_sq'] = free_agents.at[3,'blk']**2
free_agents.at[3,'tov_sq'] = free_agents.at[3,'tov']**2
free_agents.at[3,'three_fg_sq'] = free_agents.at[3,'three_fg']**2

Next, I will decide which height group Forbes falls into and compute the various interaction terms.

In [45]:
if free_agents.at[3,'height']<=76:
    free_agents.at[3,'short']=1
    free_agents.at[3,'medium']=0
    free_agents.at[3,'tall']=0
elif free_agents.at[3,'height']<=81:
    free_agents.at[3,'short']=0
    free_agents.at[3,'medium']=1
    free_agents.at[3,'tall']=0
else:
    free_agents.at[3,'short']=0
    free_agents.at[3,'medium']=0
    free_agents.at[3,'tall']=1

In [46]:
free_agents.at[3,'short:pts'] = free_agents.at[3,'short'] * free_agents.at[3,'pts']
free_agents.at[3,'short:rbs'] = free_agents.at[3,'short'] * free_agents.at[3,'rbs']
free_agents.at[3,'short:blk'] = free_agents.at[3,'short'] * free_agents.at[3,'blk']
free_agents.at[3,'short:efg_per'] = free_agents.at[3,'short'] * free_agents.at[3,'efg_per']
free_agents.at[3,'short:ast'] = free_agents.at[3,'short'] * free_agents.at[3,'ast']
free_agents.at[3,'short:stl'] = free_agents.at[3,'short'] * free_agents.at[3,'stl']
free_agents.at[3,'short:three_fg'] = free_agents.at[3,'short'] * free_agents.at[3,'three_fg']

free_agents.at[3,'tall:pts'] = free_agents.at[3,'tall'] * free_agents.at[3,'pts']
free_agents.at[3,'tall:rbs'] = free_agents.at[3,'tall'] * free_agents.at[3,'rbs']
free_agents.at[3,'tall:blk'] = free_agents.at[3,'tall'] * free_agents.at[3,'blk']
free_agents.at[3,'tall:efg_per'] = free_agents.at[3,'tall'] * free_agents.at[3,'efg_per']
free_agents.at[3,'tall:ast'] = free_agents.at[3,'tall'] * free_agents.at[3,'ast']
free_agents.at[3,'tall:stl'] = free_agents.at[3,'tall'] * free_agents.at[3,'stl']
free_agents.at[3,'tall:three_fg'] = free_agents.at[3,'tall'] * free_agents.at[3,'three_fg']

Next, I will set `RFA` and `chg_team`.

In [47]:
free_agents.at[3,'RFA'] = 0
free_agents.at[3,'chg_team'] = 1

Lastly, I will define the length and length binary features for Forbes.

In [48]:
free_agents.at[3,'length'] = 2
free_agents.at[3,'two_three_yr'] = 1
free_agents.at[3,'four_five_yr'] = 0

In [49]:
free_agents.head()

,name,height,age,min,pts,rbs,ast,stl,blk,tov,...,tall:blk,tall:efg_per,tall:ast,tall:stl,tall:three_fg,RFA,chg_team,length,two_three_yr,four_five_yr
0,Montrez Harrell,79,28.50,26.10,14.90,7.90,2.10,0.60,0.70,1.30,...,0.00,0.00,0.00,0.00,0.00,0,1,3,1,0
1,Dennis Schoder,75,28.80,33.40,17.50,3.60,4.90,0.90,0.00,2.60,...,0.00,0.00,0.00,0.00,0.00,0,1,4,0,1
2,Pat Connaughton,77,29.50,28.80,12.10,4.60,1.30,1.00,0.40,0.80,...,0.00,0.00,0.00,0.00,0.00,0,1,4,0,1
3,Bryn Forbes,74,29.00,14.30,7.20,1.20,0.70,0.40,0.00,0.90,...,0.00,0.00,0.00,0.00,0.00,0,1,2,1,0


Now, that Forbes' feature values have been calculated, I will predict his upcoming salary using the model.

In [50]:
b_forbes = [x for x in free_agents.loc[3][X].values]

In [51]:
predict(result, [b_forbes])

[array([3350926.24831904])]

The model predicts that Forbes will make 3.35 million dollars next year, which is significantly lower than Connaughton. While Forbes is playing and shooting less than Connaughton currently, this percentages suggest that he could be very productive if given more time. This might be a very good bargain for Seattle. I want to take a look at one more potential option.

### Trey Burke

Another option is Trey Burke. Burke is currently a point guard for the Dallas Mavericks and is shooting 42.3% from three so far. Here are his other statistical numbers.

Min: 11.3
<br>Pts: 6.4
<br>Rbs: 0.7
<br>Ast: 1.6
<br>Stl: 0.4
<br>Blk: 0.0
<br>Tov: 0.9
<br>Three_FG: 0.8
<br>eFG_per: 0.52

Harrell is 6'0 (72 inches) and will be 29.5 when free agency happens next summer. He will be an Unrestricted Free Agent and is said to be looking for a 1 year contract.

I will start by adding Burke to the `free_agents` DataFrame. 

In [52]:
free_agents.loc[4] = ['Trey Burke', 72, 29.5, 11.3, 6.4, 0.7, 1.6, 0.4, 0.0, 0.9, 0.8, 0.520] + 30*[0]

In [53]:
free_agents

,name,height,age,min,pts,rbs,ast,stl,blk,tov,...,tall:blk,tall:efg_per,tall:ast,tall:stl,tall:three_fg,RFA,chg_team,length,two_three_yr,four_five_yr
0,Montrez Harrell,79,28.50,26.10,14.90,7.90,2.10,0.60,0.70,1.30,...,0.00,0.00,0.00,0.00,0.00,0,1,3,1,0
1,Dennis Schoder,75,28.80,33.40,17.50,3.60,4.90,0.90,0.00,2.60,...,0.00,0.00,0.00,0.00,0.00,0,1,4,0,1
2,Pat Connaughton,77,29.50,28.80,12.10,4.60,1.30,1.00,0.40,0.80,...,0.00,0.00,0.00,0.00,0.00,0,1,4,0,1
3,Bryn Forbes,74,29.00,14.30,7.20,1.20,0.70,0.40,0.00,0.90,...,0.00,0.00,0.00,0.00,0.00,0,1,2,1,0
4,Trey Burke,72,29.50,11.30,6.40,0.70,1.60,0.40,0.00,0.90,...,0.00,0.00,0.00,0.00,0.00,0,0,0,0,0


Now, I will create the necessary features for Burke. First I will get the squared features.

In [54]:
free_agents.at[4,'pts_sq'] = free_agents.at[4,'pts']**2
free_agents.at[4,'min_sq'] = free_agents.at[4,'min']**2
free_agents.at[4,'rbs_sq'] = free_agents.at[4,'rbs']**2
free_agents.at[4,'ast_sq'] = free_agents.at[4,'ast']**2
free_agents.at[4,'stl_sq'] = free_agents.at[4,'stl']**2
free_agents.at[4,'blk_sq'] = free_agents.at[4,'blk']**2
free_agents.at[4,'tov_sq'] = free_agents.at[4,'tov']**2
free_agents.at[4,'three_fg_sq'] = free_agents.at[4,'three_fg']**2

Next, I will decide which height group Burke falls into and compute the various interaction terms.

In [55]:
if free_agents.at[4,'height']<=76:
    free_agents.at[4,'short']=1
    free_agents.at[4,'medium']=0
    free_agents.at[4,'tall']=0
elif free_agents.at[4,'height']<=81:
    free_agents.at[4,'short']=0
    free_agents.at[4,'medium']=1
    free_agents.at[4,'tall']=0
else:
    free_agents.at[4,'short']=0
    free_agents.at[4,'medium']=0
    free_agents.at[4,'tall']=1

In [56]:
free_agents.at[4,'short:pts'] = free_agents.at[4,'short'] * free_agents.at[4,'pts']
free_agents.at[4,'short:rbs'] = free_agents.at[4,'short'] * free_agents.at[4,'rbs']
free_agents.at[4,'short:blk'] = free_agents.at[4,'short'] * free_agents.at[4,'blk']
free_agents.at[4,'short:efg_per'] = free_agents.at[4,'short'] * free_agents.at[4,'efg_per']
free_agents.at[4,'short:ast'] = free_agents.at[4,'short'] * free_agents.at[4,'ast']
free_agents.at[4,'short:stl'] = free_agents.at[4,'short'] * free_agents.at[4,'stl']
free_agents.at[4,'short:three_fg'] = free_agents.at[4,'short'] * free_agents.at[4,'three_fg']

free_agents.at[4,'tall:pts'] = free_agents.at[4,'tall'] * free_agents.at[4,'pts']
free_agents.at[4,'tall:rbs'] = free_agents.at[4,'tall'] * free_agents.at[4,'rbs']
free_agents.at[4,'tall:blk'] = free_agents.at[4,'tall'] * free_agents.at[4,'blk']
free_agents.at[4,'tall:efg_per'] = free_agents.at[4,'tall'] * free_agents.at[4,'efg_per']
free_agents.at[4,'tall:ast'] = free_agents.at[4,'tall'] * free_agents.at[4,'ast']
free_agents.at[4,'tall:stl'] = free_agents.at[4,'tall'] * free_agents.at[4,'stl']
free_agents.at[4,'tall:three_fg'] = free_agents.at[4,'tall'] * free_agents.at[4,'three_fg']

Next, I will set `RFA` and `chg_team`.

In [57]:
free_agents.at[4,'RFA'] = 0
free_agents.at[4,'chg_team'] = 1

Lastly, I will define the length and length binary features for Burke.

In [58]:
free_agents.at[4,'length'] = 1
free_agents.at[4,'two_three_yr'] = 0
free_agents.at[4,'four_five_yr'] = 0

In [59]:
free_agents.head()

,name,height,age,min,pts,rbs,ast,stl,blk,tov,...,tall:blk,tall:efg_per,tall:ast,tall:stl,tall:three_fg,RFA,chg_team,length,two_three_yr,four_five_yr
0,Montrez Harrell,79,28.50,26.10,14.90,7.90,2.10,0.60,0.70,1.30,...,0.00,0.00,0.00,0.00,0.00,0,1,3,1,0
1,Dennis Schoder,75,28.80,33.40,17.50,3.60,4.90,0.90,0.00,2.60,...,0.00,0.00,0.00,0.00,0.00,0,1,4,0,1
2,Pat Connaughton,77,29.50,28.80,12.10,4.60,1.30,1.00,0.40,0.80,...,0.00,0.00,0.00,0.00,0.00,0,1,4,0,1
3,Bryn Forbes,74,29.00,14.30,7.20,1.20,0.70,0.40,0.00,0.90,...,0.00,0.00,0.00,0.00,0.00,0,1,2,1,0
4,Trey Burke,72,29.50,11.30,6.40,0.70,1.60,0.40,0.00,0.90,...,0.00,0.00,0.00,0.00,0.00,0,1,1,0,0


Now, that Burke's feature values have been calculated, I will predict his upcoming salary using the model.

In [60]:
t_burke = [x for x in free_agents.loc[4][X].values]

In [61]:
predict(result, [t_burke])

[array([627791.0703349])]

The model predicts that Burke will only be offered a contract of 628000 dollar contract. This is a little off because of the NBA salary minimums. As someone who will have been in the league for 9 seasons at the start of next year, the minimum salary Burke can make is 2401537 dollars. However, the model does show that a player producing at Burke's level will likely not sign above this minimum.

While the model had Connaughton's salary over 15 million dollars per season, I have found a couple cheaper options for the front office to consider in Bryn Forbes and Trey Burke.

In [62]:
con.close()